In [ ]:
# #NOTE: this code is only used after completing 3_EntrainmentCalculation/Entrainment_Preprocessing.ipynb

In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class

In [ ]:
####################################
#LOADING CLASSES

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)

In [ ]:
####################################
#MAKING OUTPUT DIRECTORY

In [ ]:
#OUTPUT DIRECTORIES
outputDirectory=os.path.join(mainDirectory,'Code','OUTPUT',
                             'Variable_Calculation','LagrangianArrays')
os.makedirs(outputDirectory, exist_ok=True)

#Data Output Directories
def MakeDataDirectories(outputDirectory,res,t_res,Nz_str):
    outputDataDirectory = os.path.join(outputDirectory,f"{res}_{t_res}_{Nz_str}nz",
                                       'PROCESSED_Lagrangian_Binary_Array')
    os.makedirs(outputDataDirectory, exist_ok=True)

    return outputDataDirectory

In [ ]:
####################################
#FUNCTIONS

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str

In [ ]:
# def WriteTimesteps_NetCDF(dataNC, parcelNC, timeStrings, res,t_res,Np_str,Nz_str, outputDataDirectory,outputParcelDirectory):
        
#     for count,time in tqdm(enumerate(timeStrings), total=len(timeStrings), desc="Writing timesteps"):
#         dataT = dataNC.isel(time=count)
#         parcelT = parcelNC.isel(time=count)
    
#         #making filenames
#         outputDataFile = os.path.join(outputDataDirectory, f"cm1out_{res}_{t_res}_{Nz_str}nz_{time}.nc")
#         outputParcelFile = os.path.join(outputParcelDirectory, f"cm1out_pdata_{res}_{t_res}_{Np_str}np_{time}.nc")
        
#         dataT.to_netcdf(outputDataFile, engine="h5netcdf")
#         parcelT.to_netcdf(outputParcelFile, engine="h5netcdf")

# #EXAMPLE
# WriteTimesteps_NetCDF(dataNC, parcelNC, timeStrings, res, t_res, Np_str, Nz_str,outputDataDirectory,outputParcelDirectory)

In [ ]:
def WriteTimesteps_H5(dataNC, timeStrings, res, t_res, Np_str, Nz_str,
                      outputDataDirectory):
    """
    Writes each timestep of dataNC and parcelNC to separate .h5 files,
    mirroring WriteTimestepsNetCDF but using h5py instead of xarray.to_netcdf.
    """
    from tqdm import tqdm

    for count, time in tqdm(enumerate(timeStrings), total=len(timeStrings), desc="Writing timesteps"):
        # Extract single timestep
        dataT = dataNC.isel(time=count)

        # Build file names (same as NetCDF version)
        outputDataFile = os.path.join(
            outputDataDirectory,
            f"PROCESSED_Lagrangian_Binary_Array_{res}_{t_res}_{Nz_str}nz_{time}.h5"
        )
        
        # --- Write data timestep ---
        with h5py.File(outputDataFile, "w") as f_data:
            for var_name, da in dataT.data_vars.items():
                f_data.create_dataset(
                    var_name,
                    data=da.values,
                    dtype="bool",
                    compression="gzip"
                )
                # optionally save metadata
                for attr, val in da.attrs.items():
                    f_data[var_name].attrs[attr] = val

        # print(f"Saved time {time} to {outputDataFile}","\n")

In [ ]:
def GetDataNC(ModelData):
    one = f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz"
    two = f"{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}"
    
    ds = xr.open_dataset(f"/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/{one}/Entrainment_Preprocessing/processed_binary_arrays_{two}.h5")
    
    ds = ds.rename({'phony_dim_0': 'time', 'phony_dim_1': 'p'})

    return ds

In [ ]:
####################################
#RUNNING

In [ ]:
#getting data
dataNC = GetDataNC(ModelData)

outputDataDirectory = MakeDataDirectories(outputDirectory,ModelData.res,ModelData.t_res,ModelData.Nz_str)

# running
WriteTimesteps_H5(dataNC, ModelData.timeStrings, ModelData.res, ModelData.t_res, ModelData.Np_str, ModelData.Nz_str,outputDataDirectory)